In [111]:
import requests
from bs4 import BeautifulSoup
import re
from IPython.display import Image
import time

In [112]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [453]:
# Utilizamos Selenium para extraer datos

from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
opts = Options()
opts.set_headless()
assert opts.headless  # Operating in headless mode
#browser = Firefox(options=opts)

<ipython-input-453-b56b380520ce>:6: DeprecationWarning: use setter for headless property instead of set_headless
  opts.set_headless()


In [114]:
def extract_rarity_value(text):
    text = text.replace(',','')
    data = int(re.findall('(\d+)',text)[0])
    return data
    

In [115]:
types=[]
skin=[]
total_traits =[]
traits =[]
punk_id=[]
rarity = []

total_punks = 10000

for i in range(total_punks):
    try:
        punk_id.append(i)
        browser = Firefox(executable_path = '/home/vant/Descargas/geckodriver-v0.29.1-linux64/geckodriver')
        url = 'https://defypunk.com/punks/'+str(i)
        browser.get(url)
        browser.implicitly_wait(20)

        rars = browser.find_element_by_xpath('/html/body/div/div/main/div/div/div/div[1]/div[2]')

        rarity.append(extract_rarity_value(rars.text))

        items = browser.find_element_by_xpath('/html/body/div/div/main/div/div/div/div[1]')
        # these are the items marked with tag a
        values = items.find_elements_by_tag_name('a')

        types.append(values[0].text)
        skin_property =(values[1].text).split('-')
        skin.append(skin_property[0])
        total_traits.append(values[2].text)
        traits.append(','.join([x.text for x in values[3:]]))
        browser.close()
    except Exception as err:
        print(f'exepcion {err}')
        if len(types) == len(skin) == len(total_traits) == len(traits) == len(punk_id) == len(rarity):
            df = pd.DataFrame({'id':punk_id,'types':types,'rarity':rarity,'skin':skin,'total_traits':total_traits,'traits':traits})
        else:
            print('not all have same length: investigate possible discrepancies')

In [119]:
df = pd.DataFrame({'id':punk_id,'types':types,'rarity':rarity,'skin':skin,'total_traits':total_traits,'traits':traits})
#df.set_index('id',inplace=True)

In [120]:
df.tail()

,id,types,rarity,skin,total_traits,traits
0,0,Female,2023,Mid,3,"Blonde Bob,Earring,Green Eye Shadow,Hidden Ear..."
1,1,Male,4352,Dark,2,"Mohawk,Smile"
2,2,Female,8090,Light,1,Wild Hair
3,3,Male,7896,Dark,3,"Nerd Glasses,Pipe,Wild Hair"
4,4,Male,7332,Mid,4,"Big Shades,Earring,Goat,Wild Hair"


df.to_csv('punks_atributes.csv')

In [ ]:
# srapping transactions

In [554]:
punk_id =[]
amount_eth = []
amount_dollars = []
transaction_date = []

total_punks = 5000

for j in range(total_punks):
    try:        
        browser = Firefox(executable_path = '/home/vant/Descargas/geckodriver-v0.29.1-linux64/geckodriver')
        url = 'https://www.larvalabs.com/cryptopunks/details/'+str(j)
        browser.get(url)
        browser.implicitly_wait(15)

        table = browser.find_element_by_tag_name('table')
        #rows = table.find_elements_by_tag_name('td')
        
        rows = table.find_elements_by_class_name('punk-history-row-sold')
        for row in rows:
            punk_id.append(j)
            items = row.text.split(' ')             
            amount_eth.append(items[3])
            amount_dollars.append(items[4])
            transaction_date.append(' '.join(items[5:]))
            
     
        browser.close()
        

        
    except Exception as err:
        print(f'exepcion {err}')
        
df = pd.DataFrame({'id':punk_id,'amount_eth':amount_eth,'amount_dollars':amount_dollars,'transaction':transaction_date})
        

exepcion Message: Unable to locate element: table

exepcion Message: TypeError: browsingContext.currentWindowGlobal is null



KeyboardInterrupt: 

In [556]:
len(punk_id),len(amount_eth),len(amount_dollars),len(transaction_date)

(2694, 2694, 2694, 2694)

In [557]:
df = pd.DataFrame({'id':punk_id,'amount_eth':amount_eth,'amount_dollars':amount_dollars,'transaction':transaction_date})

In [559]:
df.tail()

,id,amount_eth,amount_dollars,transaction
2689,2677,3.99Ξ,"($1,427)","Sep 28, 2020"
2690,2677,2.30Ξ,($796),"Sep 24, 2020"
2691,2679,0.31Ξ,($69),"Jul 18, 2017"
2692,2681,277Ξ,"($539,527)","Feb 19, 2021"
2693,2681,83Ξ,"($54,018)","Dec 26, 2020"


In [560]:
df.to_csv('transactions_2681.csv')